<a href="https://colab.research.google.com/github/Suprathika-vangari/INFO-5307-Project/blob/main/INFO_5307_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1:**
* Using Pandas for data manipulation and Plotly Express for visualization, both
crucial tools for data analysis in Python.
* This will enable us to handle the dataset and create insightful visualizations to tackle Walmart's inventory management issues.

In [37]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import pandas as pd

**Step 2:**
* Reading a CSV file named 'walmart.csv' using Pandas and then reducing the dataset to the first 50,000 rows before saving it as 'walmart_trim.csv'.
* Trimming the dataset might be a practical approach to start analyzing a smaller subset for faster processing or to focus on specific portions of the data. This step helped streamline the exploratory analysis by working with a manageable subset.

In [38]:
data = pd.read_csv('walmart.csv',encoding= 'latin-1')
data = data[:50000]
data.to_csv('walmart_trim.csv',index= False)

<ipython-input-38-88aeeb798a28>:1: DtypeWarning:

Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.



In [90]:
#printing columns/features from the dataset
print(data.columns)

Index(['Unnamed: 0', 'Store', 'Date', 'IsHoliday', 'Dept', 'Weekly_Sales',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size',
       'product_container', 'product_name', 'product_sub_category', 'profit',
       'region', 'sales', 'ship_date', 'ship_mode', 'shipping_cost', 'state',
       'unit_price', 'zip_code', 'day_of_week', 'month', 'year'],
      dtype='object')


**Step 3:**
* Mapping the numerical store IDs to their corresponding store names, converting the 'Date' column to a datetime format, and then deriving additional features like 'day_of_week', 'month', and 'year' from the date.
* This transformation is quite helpful as it allows for better analysis based on specific time-related trends such as sales patterns across different days, months, and years. Understanding the temporal aspects of sales data is crucial for inventory management, especially in recognizing seasonal variations and periodic trends.

In [40]:
store_mapping = {1: 'Store_1', 2: 'Store_2', 3: 'Store_3',4:'Store_4',5:'Store_5',6:'Store_6',None: 'Null'}  # Add more mappings as needed
data['Store'] = data['Store'].map(store_mapping)
data['Date'] = pd.to_datetime(data['Date'])
data['day_of_week'] = data['Date'].dt.strftime('%A')
data['month'] = data['Date'].dt.month_name()
data['year'] = data['Date'].dt.year

**Step 4:**
* Performed some significant data cleaning steps.
* Tried to refine the dataset by filtering rows based on specific criteria, such as selecting the top modes of shipping, regions, and states that have higher occurrences.
* Additionally,handled missing or erroneous values in the 'unit_price', 'sales', and 'Dept' columns by converting them to numeric types, dropping NaN values, and ensuring data consistency by converting 'sales' to float and 'Dept' to string type.

* Cleaning the data is a crucial step in preparing it for analysis. By removing irrelevant or inconsistent entries and ensuring data integrity, you're setting the foundation for more accurate and reliable insights into Walmart's inventory management.

In [41]:
print('Data Before Cleaning: {}'.format(len(data)))
# Data Cleaning
clean_data = data[data['ship_mode'].isin(list(data['ship_mode'].value_counts().keys())[:3])]
clean_data = clean_data[clean_data['region'].isin(list(data['region'].value_counts().keys())[:4])]
clean_data = clean_data[clean_data['state'].isin(list(data['state'].value_counts().keys())[:50])] # considering top 40 states
numeric_column = pd.to_numeric(clean_data['unit_price'], errors='coerce')
clean_data = clean_data[numeric_column.notna()]
numeric_column2 = pd.to_numeric(clean_data['sales'], errors='coerce')
clean_data = clean_data[numeric_column2.notna()]
clean_data = clean_data[clean_data['Dept']!='nan']
clean_data.dropna(subset=['Dept'], inplace=True)
clean_data['sales'] = clean_data['sales'].astype(float)
clean_data['Dept'] = clean_data['Dept'].astype(str)
print('Data After Cleaning: {}'.format(len(clean_data)))


Data Before Cleaning: 50000
Data After Cleaning: 48404


**Step 5:**
* Selected specific columns relevant to inventory management and created a subset named 'inventory_data'. Truncating the 'product_name' column to the first 25 characters could facilitate easier analysis and visualization. Additionally, isolated data for the years 2011 and 2012 in the 'yearly_data' subset, possibly for analyzing trends or patterns over these years.

* By narrowing down to essential columns and focusing on specific years, I'm aiming for more targeted insights into inventory-related trends and sales patterns during those periods. This refined dataset will likely streamline the subsequent exploratory analysis for inventory management solutions

In [81]:
inventory_related_columns = ['Date','Dept', 'Weekly_Sales', 'product_container', 'product_name', 'product_sub_category', 'sales', 'ship_date', 'ship_mode', 'unit_price','Store','day_of_week','month','year','state','region','zip_code','profit', 'MarkDown1']
inventory_data = clean_data[inventory_related_columns]
inventory_data['product_name'] = inventory_data['product_name'].str.slice(0, 25)
yearly_data = inventory_data[(inventory_data['year']==2012)|(inventory_data['year']==2011)] # for monthly usage purpose
inventory_data.head()

<ipython-input-81-4179f727b051>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Dept,Weekly_Sales,product_container,product_name,product_sub_category,sales,ship_date,ship_mode,unit_price,Store,day_of_week,month,year,state,region,zip_code,profit,MarkDown1
0,2010-02-05,1.0,24924.50,Jumbo Drum,Global Enterprise Series,Chairs & Chairmats,21.84,3/2/2020,Delivery Truck,3.29,Store_1,Friday,February,2010,Wisconsin,Central,54481,19559.92268,NaN
1,2010-02-05,26.0,11737.12,Jumbo Drum,Global Enterprise Series,Chairs & Chairmats,1811.67,3/7/2020,Delivery Truck,258.98,Store_1,Friday,February,2010,Wisconsin,Central,54481,19559.92268,NaN
2,2010-02-05,17.0,13223.76,Jumbo Box,Bevis Rectangular Confere,Tables,6129.06,11/15/2021,Delivery Truck,145.98,Store_1,Friday,February,2010,Texas,Central,76051,7535.9388,NaN
3,2010-02-05,45.0,37.44,Small Box,Xerox 1923,Paper,198.90,8/8/2020,Regular Air,6.68,Store_1,Friday,February,2010,Arizona,West,85281,18860.92419,NaN
4,2010-02-05,28.0,1085.29,Small Box,SAFCO Mobile Desk Side Fi,Storage & Organization,1875.28,8/18/2021,Express Air,42.76,Store_1,Friday,February,2010,Florida,South,33063,24750.4921,NaN


In [55]:
inventory_data.shape

(48404, 18)

In [63]:
product_counts = inventory_data['product_name'].value_counts().head(20)

fig = px.bar(product_counts, x=product_counts.index, y=product_counts.values,
labels={'x': 'Product Name', 'y': 'Count'},
title='Top 10 Products by Count')

fig.update_layout(xaxis_title_text='Product Name', yaxis_title_text='Count')
fig.show()

In [74]:
inventory_data['profit'] = inventory_data['profit'].astype(float)
category_profit_counts = inventory_data.groupby(['product_sub_category']).agg({'profit':'sum'}).reset_index()
px.line(category_profit_counts,x='product_sub_category',y='profit',markers=True,title= 'Sales vs Department Graph')

<ipython-input-74-cc0128b8b8da>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
yearly_data['month'] = pd.Categorical(yearly_data['month'], categories=month_order, ordered=True)
shipping_month_counts = yearly_data.groupby(['month','ship_mode']).agg({'sales':'count'}).reset_index()
shipping_month_counts.columns = ['month','ship_mode','count']
fig = px.scatter(shipping_month_counts, x="month", y="count", color="ship_mode",
                 size='count',title= 'Shipping Mode vs Month Analysis')
fig.show()

<ipython-input-61-6b07ad72d486>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
store_counts = inventory_data.groupby(['Store','product_sub_category']).agg({'sales':'sum'}).reset_index()
store_counts.columns = ['Store','product_sub_category','sales']
fig = px.bar(store_counts, x="Store", y="sales", color="product_sub_category", title="Store-Category counts", barmode='group')
fig.show()

In [33]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
yearly_data['month'] = pd.Categorical(yearly_data['month'], categories=month_order, ordered=True)
yearly_data['sales']= yearly_data['sales'].astype(float)
store_month_counts = yearly_data.groupby(['Store','month']).agg({'sales':'sum'}).reset_index()
store_month_counts.columns = ['Store','month','count']
fig = px.line(store_month_counts, x="month", y="count", color="Store",category_orders={'month': month_order},markers = True,title= 'Month-Store Analysis')
fig.update_traces(textposition="bottom right")
fig.show()

<ipython-input-33-8d72e8a6f79f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-8d72e8a6f79f>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [69]:
store_counts = inventory_data.groupby(['state','product_sub_category']).agg({'sales':'sum'}).reset_index()
store_counts.columns = ['state','product_sub_category','sales']
fig = px.bar(store_counts, x="state", y="sales", color="product_sub_category", title="State-Sub Catergory counts")
fig.show()

In [35]:
inventory_data['sales'] = inventory_data['sales'].astype(float)
inventory_data['Dept'] = inventory_data['Dept'].astype(str)
state_dept = inventory_data.groupby(['state','Dept']).agg({'sales':'sum'}).reset_index()
fig = px.scatter(state_dept, x="state", y="sales", color="Dept",title='State/Dept Sales')
fig.show()


<ipython-input-35-96e4e20f56ab>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-35-96e4e20f56ab>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
inventory_data['Dept'] = inventory_data['Dept'].astype(str)
inventory_data['sales']= inventory_data['sales'].astype(float)
store_month_counts = inventory_data.groupby(['Dept']).agg({'sales':'sum'}).reset_index()
px.line(store_month_counts,x='Dept',y='sales',markers=True,title= 'Sales vs Department Graph')

<ipython-input-14-6c277d765d0c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-14-6c277d765d0c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Which product categories consistently exhibit high sales, indicating a need for increased inventory stocking to maximize revenue?

Paper, Office Furnishings

Are there identifiable periods or seasons when certain product categories experience heightened sales, necessitating adjusted inventory levels to prevent stockouts?

We see that in the month of June : Binders and Binder accessories have a big spike

Which stores have the highest sales for a particular product category?

First position is taken by Office Machine because of its high pricing in all stores. Second and third positions gets shuffled in between telephones & communication and tables

Are there any seasonal trends in sales at the store level?

March tends to be the month with highest sales irrespective of stores

Which departments generate the most sales revenue for Walmart?

Dept 72

Which states or regions have the highest sales for a particular product category?

Office Machine stands as the most revenue generating in every state. the second position keeps varying .For examples , texas has telephones as second while new jersey has tables.




In [86]:
# changing values to generate answers for above questions
store_counts = yearly_data.groupby(['month','product_sub_category']).agg({'sales':'sum'}).reset_index()
store_counts.columns = ['month','product_sub_category','sales']
fig = px.bar(store_counts, x="month", y="sales", color="product_sub_category", title="Month-Sub Category counts")
fig.show()


In [77]:
region_category_sales = inventory_data.groupby(['region', 'product_sub_category']).agg({'sales': 'sum'}).reset_index()
top_regions = region_category_sales.groupby('region')['sales'].sum().nlargest(5).index
top_categories = region_category_sales.groupby('product_sub_category')['sales'].sum().nlargest(10).index

filtered_data = region_category_sales[region_category_sales['region'].isin(top_regions) & region_category_sales['product_sub_category'].isin(top_categories)]

fig = px.bar(filtered_data, x="region", y="sales", color="product_sub_category", title="Regional Sales by Product Categories", barmode='group')
fig.show()

In [84]:
seasonal_sales = inventory_data.groupby(['month', 'product_sub_category']).agg({'sales': 'sum'}).reset_index()
top_categories = seasonal_sales.groupby('product_sub_category')['sales'].sum().nlargest(5).index

filtered_data = seasonal_sales[seasonal_sales['product_sub_category'].isin(top_categories)]

fig = px.line(filtered_data, x="month", y="sales", color="product_sub_category", title="Seasonal Sales Trends for Top Categories")
fig.show()
